### Create Component

In [22]:
from kfp.dsl import component, pipeline

@component
def adding(a: int, b: int)->str:
    return str(a+b)

@component
def diff(a: str, b: int)->int:
    return int(a)-b

### Create Pipeline

In [23]:
@pipeline(name='kfp-lightw-pipe')
def pipeline(a: int, b: int):
    sum_job = adding(a=a,b=b)
    diff_job = diff(a=sum_job.output, b=b)

### Compile Pipeline

In [24]:
from kfp import compiler

compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path='pipeline.yaml')

### Create Template and Upload Pipeline to Registry

In [25]:
from kfp.registry import RegistryClient

client = RegistryClient(host=f"https://us-central1-kfp.pkg.dev/vtxdemos/kfp-repo")

In [26]:
templateName, versionName = client.upload_pipeline(
  file_name="pipeline.yaml",
  tags=["v1", "latest"]
  )

### Run Pipeline

In [27]:
from google.cloud import aiplatform

aiplatform.init(project="vtxdemos", staging_bucket="gs://vtxdemos-tmp")

job = aiplatform.PipelineJob(
    display_name="hello-world-latest",
    template_path="https://us-central1-kfp.pkg.dev/vtxdemos/kfp-repo/kfp-lightw-pipe/v1",
    parameter_values={
        "a":10,
        "b":5
    })

In [28]:
job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/254356041555/locations/us-central1/pipelineJobs/kfp-lightw-pipe-20230524175341
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/254356041555/locations/us-central1/pipelineJobs/kfp-lightw-pipe-20230524175341')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/kfp-lightw-pipe-20230524175341?project=254356041555
